## 1. 구글 드라이브 연결
자세 사진 데이터를 저장한 폴더인 posture 폴더를 구글 드라이브에 올려서 연결합니다.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 2. 데이터 전처리
구글 드라이브의 posture 폴더 안에 있는 good, bad 폴더들로 라벨을 만들고 Training, Test, Val 3 세트의 NumPy 배열로 추출.

In [ ]:
!pip install mediapipe tqdm
import numpy as np
from tqdm import tqdm
import cv2
import mediapipe as mp
import os
import glob

BASE_DIR = "/content/drive/MyDrive/posture/"

# MediaPipe Pose 모델 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, model_complexity=1)

def extract_landmarks(image_path):
    """이미지 경로를 받아 (33, 4) 랜드마크 배열 반환 """
    if not os.path.exists(image_path): return None

    image = cv2.imread(image_path)
    if image is None: return None

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if not results.pose_landmarks: return None

    landmarks_object = results.pose_landmarks
    landmark_array = np.array([[lm.x, lm.y, lm.z, lm.visibility]
                              for lm in landmarks_object.landmark])
    return landmark_array

# === 데이터셋 저장용 딕셔너리 초기화 ===
# 각 세트(Train, Val, Test) 별로 X, Y 데이터를 담을 리스트 생성
dataset = {
    'train': {'X': [], 'Y': []},
    'val':   {'X': [], 'Y': []},
    'test':  {'X': [], 'Y': []}
}

# 폴더 이름을 good (바른 자세), bad (나쁜 자세)로 설정
# 각 폴더에 맞는 라벨 설정
POSTURE_FOLDERS = {
    "good": 1,
    "bad": 0
}

print(f"데이터셋 생성을 시작합니다... (경로: {BASE_DIR})")

# === 폴더 분류 및 데이터 처리 로직 ===
for label_name, label_idx in POSTURE_FOLDERS.items():
    class_path = os.path.join(BASE_DIR, label_name)

    if not os.path.isdir(class_path):
        print(f"Warning: '{class_path}' 폴더가 없습니다.")
        continue

    # 해당 클래스(good/bad) 내의 모든 하위 폴더 목록 가져오기
    # 숨김 파일 제외하고 폴더만 리스트업
    subfolders = [os.path.join(class_path, d) for d in os.listdir(class_path)
                  if os.path.isdir(os.path.join(class_path, d))]

    subfolders.sort()

    # 폴더 개수 체크 (최소 3개는 있어야 Train/Val/Test 분할 가능)
    if len(subfolders) < 3:
        print(f"오류: '{label_name}' 안의 폴더가 {len(subfolders)}개 뿐입니다. 최소 3개 이상의 폴더가 필요합니다.")
        continue

    # 폴더 역할 분배 (Test 1개, Val 1개, 나머지 Train)
    # 리스트의 맨 앞 2개를 각각 Test, Val로 쓰고 나머지를 Train으로 사용.
    test_folders = [subfolders[0]]
    val_folders  = [subfolders[1]]
    train_folders = subfolders[2:]

    # 분배된 정보를 딕셔너리로 묶어서 처리
    split_info = {
        'test': test_folders,
        'val': val_folders,
        'train': train_folders
    }

    print(f"\n=== [{label_name}] 폴더 분배 결과 ===")
    print(f"  - Test 폴더 ({len(test_folders)}개): {[os.path.basename(f) for f in test_folders]}")
    print(f"  - Val  폴더 ({len(val_folders)}개): {[os.path.basename(f) for f in val_folders]}")
    print(f"  - Train 폴더 ({len(train_folders)}개): {[os.path.basename(f) for f in train_folders]}")

    # 각 세트별로 이미지 처리 시작
    for split_name, folders in split_info.items():
        print(f"  >> '{split_name}' 데이터 처리 중...")

        for folder_path in folders:
            # 해당 폴더 내의 모든 이미지 파일 찾기 (jpg, png 등)
            # glob을 사용하여 해당 폴더 직속 파일만 가져온다
            image_files = []
            for ext in ('*.jpg', '*.jpeg', '*.png'):
                image_files.extend(glob.glob(os.path.join(folder_path, '**', ext), recursive=True))

            # 진행률 표시 (tqdm)
            for image_path in tqdm(image_files, desc=f"    {os.path.basename(folder_path)}", leave=False):
                landmarks = extract_landmarks(image_path)

                # 랜드마크 추출 성공 시 데이터 추가
                if landmarks is not None:
                    if landmarks.shape == (33, 4):
                        dataset[split_name]['X'].append(landmarks)
                        dataset[split_name]['Y'].append(label_idx)


print("\n--- 모든 처리 완료, NumPy 변환 및 저장 ---")

# === 결과 저장 ===
# 각 세트별로 numpy array로 변환하고 저장
for split_name in ['train', 'val', 'test']:
    X_arr = np.array(dataset[split_name]['X'])
    Y_arr = np.array(dataset[split_name]['Y'])

    print(f"[{split_name.upper()} SET] X: {X_arr.shape}, Y: {Y_arr.shape}")

    if len(X_arr) > 0:
        np.save(f'X_{split_name}.npy', X_arr)
        np.save(f'Y_{split_name}.npy', Y_arr)
        print(f" -> {split_name} 데이터 저장 완료")
    else:
        print(f" -> {split_name} 데이터가 비어있어 저장하지 않았습니다.")

print("\n최종 완료!")

데이터셋 생성을 시작합니다... (경로: /content/drive/MyDrive/posture/)

=== [good] 폴더 분배 결과 ===
  - Test 폴더 (1개): ['1']
  - Val  폴더 (1개): ['2']
  - Train 폴더 (7개): ['3', '4', '5', '6', '7', '8', '9']
  >> 'test' 데이터 처리 중...


  >> 'val' 데이터 처리 중...


  >> 'train' 데이터 처리 중...



=== [bad] 폴더 분배 결과 ===
  - Test 폴더 (1개): ['1']
  - Val  폴더 (1개): ['10']
  - Train 폴더 (12개): ['11', '12', '13', '14', '2', '3', '4', '5', '6', '7', '8', '9']
  >> 'test' 데이터 처리 중...


  >> 'val' 데이터 처리 중...


  >> 'train' 데이터 처리 중...



--- 모든 처리 완료, NumPy 변환 및 저장 ---
[TRAIN SET] X: (2139, 33, 4), Y: (2139,)
 -> train 데이터 저장 완료
[VAL SET] X: (212, 33, 4), Y: (212,)
 -> val 데이터 저장 완료
[TEST SET] X: (250, 33, 4), Y: (250,)
 -> test 데이터 저장 완료

최종 완료!


### 추가: 영상에서 사진 추출
자세 데이터를 위해 찍은 영상을 사진으로 추출.

In [ ]:
import cv2
import os
import shutil

# === 설정 ===
video_path = '영상 이름.mp4'   # 영상의 파일명
output_folder = 'extracted_frames'
target_image_count = 100      # 목표: 약 100장 추출
# ===========

# 1. 기존 폴더가 있다면 삭제하고 새로 만들기 (초기화)
if os.path.exists(output_folder):
    shutil.rmtree(output_folder)  # 폴더 안의 파일까지 통째로 삭제
    print(f"기존 '{output_folder}' 폴더를 삭제했습니다.")

os.makedirs(output_folder) # 폴더 새로 생성
print(f"새로운 '{output_folder}' 폴더를 생성했습니다.")


# 2. 영상 읽기 및 프레임 계산
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("영상을 열 수 없습니다. 파일명을 확인해주세요.")
else:
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # 100장을 맞추기 위한 간격 계산
    step = total_frames // target_image_count
    if step == 0: step = 1

    print(f"전체 {total_frames} 프레임 중 약 {step} 프레임마다 하나씩 저장합니다.")

    count = 0
    saved_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # 계산된 간격마다 저장
        if count % step == 0:
            filename = f"{output_folder}/img_{str(saved_count).zfill(4)}.jpg"
            cv2.imwrite(filename, frame)
            saved_count += 1

        count += 1

    cap.release()
    print(f"작업 끝! 총 {saved_count}장의 사진이 새로 저장되었습니다.")

import shutil
from google.colab import files

# 폴더 압축 (zip 파일 생성)
zip_filename = 'frames.zip'
shutil.make_archive('frames', 'zip', output_folder)

# 다운로드 실행
files.download(zip_filename)

기존 'extracted_frames' 폴더를 삭제했습니다.
새로운 'extracted_frames' 폴더를 생성했습니다.
전체 310 프레임 중 약 3 프레임마다 하나씩 저장합니다.
✅ 작업 끝! 총 104장의 사진이 새로 저장되었습니다.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>